# **Weather and Crime Relationship in Houston**
* Datasets are provided in 3 different formats: mmmyy.xls, mm-yyyy.NIBRS_Public_Data_Group_A&B.xlsx, NIBRSPublicViewMonyy.xlsx
    * Ex: sep17.xls, 11-2018.NIBRS_Public_Data_Group_A&B.xlsx, NIBRSPublicViewDec21.xlsx
*
* Merge 113 files into 3 files by format
* 'jan2010_may2018.csv', 'jun2018_dec2018.csv', and 'jan2019_jul2023.csv' will be created
* Cleaning and merging 3 files into 1 will be done with clean_merge_crime_houston_datasets.ipynb


### **Data Source**
* City of Houston
* https://www.houstontx.gov/police/cs/Monthly_Crime_Data_by_Street_and_Police_Beat.htm


### **1. Import Libraries**

In [ ]:
# !pip install xlrd

In [ ]:
#### Import the libraries needed
import pickle
import dill
import json
import glob
import os
from pathlib import Path
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### **2. Set up the Environment**

In [ ]:
# Set pd.options to add slide bars
pd.set_option('display.max_rows', 10)       # Display only 10  rows
pd.set_option('display.max_columns', 10)    # Display only 10  columns
pd.set_option("display.max_rows", None)     # Add a vertical scroll bar
pd.set_option('display.max_columns', None)  # Add a horizontal scroll bar

In [ ]:
# Set default title color
plt.style.use('fivethirtyeight')
# Set default title color
plt.rcParams.update({'text.color': "#333333", 'axes.labelcolor': "#333333"})

# set default plt figure size
plt.rcParams["figure.figsize"] = [10, 5]
# figure suptitle
plt.rcParams["figure.titlesize"] = 26
plt.rcParams["figure.titleweight"] = "bold"
# set default plt font size
# plt.rcParams["font.size"] = 24
# font weight
# plt.rcParams["font.weight"] = "bold"
# title location
plt.rcParams["axes.titlelocation"] = "left"
# title size
plt.rcParams["axes.titlesize"] = 18
# title wight
plt.rcParams["axes.titleweight"] = "bold"
# plt.rcParams["axes.labelweight"] = "bold"
# spines
plt.rcParams["axes.spines.bottom"] = True
plt.rcParams["axes.spines.left"] = True
# axis labels
# plt.rcParams["xaxis.labellocation"] = "left"
# plt.rcParams["yaxis.labellocation"] = "top"
plt.rcParams["axes.labelweight"] = "bold"

plt.rcParams.update({'text.color': "#333333", 'axes.labelcolor': "#333333", 'axes.labelweight': 'bold'})
plt.rcParams["font.weight"] = "bold"

### **3. Load Data**

In [ ]:
# Current working directory
PROJECT_ROOT_PATH = Path.cwd()

# Define the data folder path
RAW_DATA_PATH = os.path.join(PROJECT_ROOT_PATH, 'data')

In [ ]:
# Open 101 xls files and combine them into one dataframe.
df = pd.DataFrame()
col_names_df = ['Occurrence Date', 'Occurrence Hour', 'NIBRS Description', 'Beat', 'Premise', 'Block Range',
                'Street Name', 'Street Type', 'Suffix', 'Offense Count']
for file in glob.glob('data/*.xls'):
    if os.path.splitext(file)[1] != 'pdf' and os.path.isfile(file):
        df = pd.concat([df, pd.read_excel(file, dtype={'Beat': str, 'Block Range':str})])

WARNING *** file size (3078551) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (2752234) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [ ]:
# Open 12 xlsx files and combine them into one dataframe.
df2 = pd.DataFrame()
df3 = pd.DataFrame()
col_names_df2 = ['Occurrence Date', 'Occurrence Hour', 'NIBRS Description' 'Offense Count', 'Beat', 'Premise', 'Block Range',
                 'Street Name', 'Street Type', 'Suffix']
col_names_df3 = ['Incident', 'Occurrence Date', 'Occurrence Hour', 'NIBRS Class', 'NIBRS Description', 'Offense Count', 'Beat',
                 'Premise', 'Block Range', 'Street Name', 'Street Type', 'Suffix', 'City', 'ZIP Code', 'RMSOccurrenceDate',
                 'RMSOccurrenceHour', 'NIBRS Class', 'Offense Count', 'Street No', 'Street Type', 'ZIP Code', 'Map Longitude', 'Map Latitude']

for file in glob.glob('data/*.xlsx'):
    if os.path.splitext(file)[1] != 'pdf' and os.path.isfile(file):
        result = re.match(r"[0-9]{2}-[0-9]{4}", os.path.basename(file))
        if  result:
            df2 = pd.concat([df2, pd.read_excel(file, skiprows=11, index_col=0, dtype={'Beat': str, 'Block Range':str})])
        else:
            df3 = pd.concat([df3, pd.read_excel(file, dtype={'Beat': str})])

### **4. Check Dataframes**

In [ ]:
print("January 2010 to May 2018 --> jan2010_may2018.csv", df.shape)
print("June 2018 to December 2018 --> jun2018_dec2018.csv", df2.shape)
print("January 2019 to July 2023 --> jan2019_jul2023.csv", df3.shape)

January 2010 to May 2018 --> jan2010_may2018.csv (1053346, 18)
June 2018 to December 2018 --> jun2018_dec2018.csv (145713, 17)
January 2019 to July 2023 --> jan2019_jul2023.csv (1111335, 23)


In [ ]:
df.head()

,Date,Hour,Offense Type,Beat,Premise,Block Range,Street Name,Type,Suffix,Offenses,BlockRange,StreetName,# offenses,# Of Offenses,# Offenses,# Of
0,01/16/2017,5,Theft,10H10,Commercial Parking Lot or Garage,6000-6099,CANAL,ST,-,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,03/31/2017,18,Rape,10H10,Residence or House,200-299,N LENOX ST,ST,-,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,09/03/2017,19,Burglary,10H10,Apartment,1900-1999,RUNNELS,-,-,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,09/03/2017,21,Aggravated Assault,10H10,"Road, Street, or Sidewalk",2000-2099,RUNNELS,-,-,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,09/04/2017,13,Aggravated Assault,10H10,"Road, Street, or Sidewalk",400-499,LOCKWOOD DR,-,-,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1053346 entries, 0 to 10250
Data columns (total 18 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   Date           1053341 non-null  object 
 1   Hour           1053345 non-null  object 
 2   Offense Type   1053345 non-null  object 
 3   Beat           1053329 non-null  object 
 4   Premise        1019749 non-null  object 
 5   Block Range    718504 non-null   object 
 6   Street Name    718660 non-null   object 
 7   Type           1053345 non-null  object 
 8   Suffix         1053345 non-null  object 
 9   Offenses       126364 non-null   float64
 10  BlockRange     334684 non-null   object 
 11  StreetName     334684 non-null   object 
 12  # offenses     334684 non-null   float64
 13  # Of Offenses  574365 non-null   float64
 14  Unnamed: 1     0 non-null        float64
 15  # Offenses     10319 non-null    float64
 16  # Of           7613 non-null     float64
 17  Field11   

In [ ]:
df_columns = df.columns
df_columns

Index(['Date', 'Hour', 'Offense Type', 'Beat', 'Premise', 'Block Range',
       'Street Name', 'Type', 'Suffix', 'Offenses', 'BlockRange', 'StreetName',
       '# offenses', '# Of Offenses', 'Unnamed: 1', '# Offenses', '# Of',
       'Field11'],
      dtype='object')

In [ ]:
print("----- Null Count -----")
print(df.isna().sum())

----- Null Count -----
Date                   5
Hour                   1
Offense Type           1
Beat                  17
Premise            33597
Block Range       334842
Street Name       334686
Type                   1
Suffix                 1
Offenses          926982
BlockRange        718662
StreetName        718662
# offenses        718662
# Of Offenses     478981
# Offenses       1043027
# Of             1045733
dtype: int64


In [ ]:
print("----- Null Ratio (%) -----")
print(df.isna().sum()/len(df)*100)

----- Null Ratio (%) -----
Date              0.000475
Hour              0.000095
Offense Type      0.000095
Beat              0.001614
Premise           3.189550
Block Range      31.788415
Street Name      31.773605
Type              0.000095
Suffix            0.000095
Offenses         88.003562
BlockRange       68.226585
StreetName       68.226585
# offenses       68.226585
# Of Offenses    45.472333
# Offenses       99.020360
# Of             99.277256
dtype: float64


In [ ]:
df2.head()

,Occurrence Date,Unnamed: 2,Occurrence Hour,Unnamed: 4,NIBRS Description,Unnamed: 6,Offense Count,Beat,Premise,Block Range,Unnamed: 11,Street Name,Unnamed: 13,Unnamed: 14,Street Type,Unnamed: 16,Suffix
NaN,2018-09-01,NaN,0,NaN,"Credit card, ATM fraud",NaN,1,5F30,Auto Dealership New/Used,12800-12899,NaN,HEMPSTEAD,NaN,NaN,RD,NaN,NaN
NaN,2018-09-01,NaN,0,NaN,Intimidation,NaN,1,8C30,"Residence, Home (Includes Apartment)",8100-8199,NaN,SNOWDEN,NaN,NaN,NaN,NaN,NaN
NaN,2018-09-01,NaN,0,NaN,Weapon law violations,NaN,1,14D10,"Parking Lot, Garage",6600-6699,NaN,MADRID,NaN,NaN,ST,NaN,NaN
NaN,2018-09-01,NaN,0,NaN,Trespass of real property,NaN,1,10H40,"Service, Gas Station",2100-2199,NaN,FANNIN,NaN,NaN,ST,NaN,NaN
NaN,2018-09-01,NaN,0,NaN,Driving under the influence,NaN,1,9C20,"Highway, Road, Street, Alley",6100-6199,NaN,EAST,NaN,NaN,FWY,NaN,NaN


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 145713 entries, nan to nan
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Occurrence Date    145713 non-null  object 
 1   Unnamed: 2         0 non-null       float64
 2   Occurrence Hour    145713 non-null  int64  
 3   Unnamed: 4         0 non-null       float64
 4   NIBRS Description  145713 non-null  object 
 5   Unnamed: 6         0 non-null       float64
 6   Offense Count      145713 non-null  int64  
 7   Beat               145564 non-null  object 
 8   Premise            145713 non-null  object 
 9   Block Range        144823 non-null  object 
 10  Unnamed: 11        0 non-null       float64
 11  Street Name        145713 non-null  object 
 12  Unnamed: 13        0 non-null       float64
 13  Unnamed: 14        0 non-null       float64
 14  Street Type        133976 non-null  object 
 15  Unnamed: 16        0 non-null       float64
 16  S

In [ ]:
df2_columns = df2.columns
df2_columns

Index(['Occurrence Date', 'Unnamed: 2', 'Occurrence Hour', 'Unnamed: 4',
       'NIBRS Description', 'Unnamed: 6', 'Offense Count', 'Beat', 'Premise',
       'Block Range', 'Unnamed: 11', 'Street Name', 'Unnamed: 13',
       'Unnamed: 14', 'Street Type', 'Unnamed: 16', 'Suffix'],
      dtype='object')

In [ ]:
col_names_df2 = ['Occurrence Date', 'Occurrence Hour', 'NIBRS Description' 'Offense Count', 'Beat', 'Premise',
 'Block Range', 'Street Name', 'Street Type', 'Suffix']

In [ ]:
print("----- Null Ratio (%) -----")
print(df2.isna().sum()/len(df2)*100)

Null Count: Occurrence Date           0
Occurrence Hour           0
NIBRS Description         0
Offense Count             0
Beat                    149
Premise                   0
Block Range             890
Street Name               0
Street Type           11737
Suffix               125181
dtype: int64


In [ ]:
df3.head()

,Incident,Occurrence\nDate,Occurrence\nHour,NIBRS\nClass,NIBRSDescription,Offense\nCount,Beat,Premise,Block Range,StreetName,Street\nType,Suffix,City,ZIP Code,RMSOccurrenceDate,RMSOccurrenceHour,NIBRSClass,OffenseCount,StreetNo,StreetType,ZIPCode,MapLongitude,MapLatitude
0,5619,2019-01-01,0.0,290,"Destruction, damage, vandalism",1.0,9C30,"Residence, Home (Includes Apartment)",9622,SAN CARLOS,NaN,NaN,HOUSTON,77013,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17319,2019-01-01,0.0,35A,"Drug, narcotic violations",1.0,7C10,"Highway, Road, Street, Alley",NaN,EAST,FWY,NaN,HOUSTON,77020,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18119,2019-01-01,0.0,290,"Destruction, damage, vandalism",1.0,16E40,"Residence, Home (Includes Apartment)",16718,LONE QUAIL,CT,NaN,HOUSTON,77489,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19019,2019-01-01,0.0,520,Weapon law violations,1.0,NaN,"Residence, Home (Includes Apartment)",1909,MELBOURNE,NaN,NaN,HOUSTON,77026-0000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20519,2019-01-01,0.0,13A,Aggravated Assault,1.0,15E30,"Residence, Home (Includes Apartment)",4034,OSBY,DR,NaN,HOUSTON,77025,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1111335 entries, 0 to 147498
Data columns (total 23 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   Incident           1111335 non-null  int64         
 1   Occurrence
Date    466945 non-null   datetime64[ns]
 2   Occurrence
Hour    466945 non-null   float64       
 3   NIBRS
Class        466945 non-null   object        
 4   NIBRSDescription   1111335 non-null  object        
 5   Offense
Count      466945 non-null   float64       
 6   Beat               1110550 non-null  object        
 7   Premise            1111334 non-null  object        
 8   Block Range        465475 non-null   object        
 9   StreetName         1111335 non-null  object        
 10  Street
Type        429802 non-null   object        
 11  Suffix             162878 non-null   object        
 12  City               1111335 non-null  object        
 13  ZIP Code           461478 no

In [ ]:
df3_columns = df3.columns
df3_columns

Index(['Incident', 'Occurrence\nDate', 'Occurrence\nHour', 'NIBRS\nClass',
       'NIBRSDescription', 'Offense\nCount', 'Beat', 'Premise', 'Block Range',
       'StreetName', 'Street\nType', 'Suffix', 'City', 'ZIP Code',
       'RMSOccurrenceDate', 'RMSOccurrenceHour', 'NIBRSClass', 'OffenseCount',
       'StreetNo', 'StreetType', 'ZIPCode', 'MapLongitude', 'MapLatitude'],
      dtype='object')

In [ ]:
col_names_df3 = ['Incident', 'Occurrence Date', 'Occurrence Hour', 'NIBRS Class',
       'NIBRS Description', 'Offense Count', 'Beat', 'Premise', 'Block Range',
       'Street Name', 'Street Type', 'Suffix', 'City', 'ZIP Code',
       'RMSOccurrenceDate', 'RMSOccurrenceHour', 'NIBRS Class', 'Offense Count',
       'Street No', 'Street Type', 'ZIP Code', 'Map Longitude', 'Map Latitude']

In [ ]:
print("----- Null Ratio (%) -----")
print(df3.isna().sum()/len(df3)*100)

Null Count: Incident                  0
Occurrence\nDate     644390
Occurrence\nHour     644390
NIBRS\nClass         644390
NIBRSDescription          0
Offense\nCount       644390
Beat                    785
Premise                   1
Block Range          645860
StreetName                0
Street\nType         681533
Suffix               948457
City                      0
ZIP Code             649857
RMSOccurrenceDate    466945
RMSOccurrenceHour    466945
NIBRSClass           466945
OffenseCount         466945
StreetNo             468920
StreetType           515353
ZIPCode              475000
MapLongitude         719188
MapLatitude          719188
dtype: int64


### **5. Clean Dataframes**

#### Drop all Null Value Columns
* These columns are automatically created when loading Excel files

In [ ]:
# Drop columns that have all nulls
df.drop(['Unnamed: 1', 'Field11'], axis=1, inplace=True)
df2.drop(['Unnamed: 2', 'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 11', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 16'], axis=1, inplace=True)

In [ ]:
# Check df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1053346 entries, 0 to 10250
Data columns (total 16 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   Date           1053341 non-null  object 
 1   Hour           1053345 non-null  object 
 2   Offense Type   1053345 non-null  object 
 3   Beat           1053329 non-null  object 
 4   Premise        1019749 non-null  object 
 5   Block Range    718504 non-null   object 
 6   Street Name    718660 non-null   object 
 7   Type           1053345 non-null  object 
 8   Suffix         1053345 non-null  object 
 9   Offenses       126364 non-null   float64
 10  BlockRange     334684 non-null   object 
 11  StreetName     334684 non-null   object 
 12  # offenses     334684 non-null   float64
 13  # Of Offenses  574365 non-null   float64
 14  # Offenses     10319 non-null    float64
 15  # Of           7613 non-null     float64
dtypes: float64(5), object(11)
memory usage: 136.6+ MB


In [ ]:
# Check df2
df2.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 145713 entries, nan to nan
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Occurrence Date    145713 non-null  object
 1   Occurrence Hour    145713 non-null  int64 
 2   NIBRS Description  145713 non-null  object
 3   Offense Count      145713 non-null  int64 
 4   Beat               145564 non-null  object
 5   Premise            145713 non-null  object
 6   Block Range        144823 non-null  object
 7   Street Name        145713 non-null  object
 8   Street Type        133976 non-null  object
 9   Suffix             20532 non-null   object
dtypes: int64(2), object(8)
memory usage: 12.2+ MB


#### Check Duplicates

In [ ]:
# Check duplicates
print("January 2010 to May 2018 --> jan2010_may2018.csv", df.duplicated().sum())
print("June 2018 to December 2018 --> jun2018_dec2018.csv", df2.duplicated().sum())
print("January 2019 to July 2023 --> jan2019_jul2023.csv", df3.duplicated().sum())

January 2010 to May 2018 --> jan2010_may2018.csv 3569
June 2018 to December 2018 --> jun2018_dec2018.csv 658
January 2019 to July 2023 --> jan2019_jul2023.csv 0


In [ ]:
df.head(3)

,Date,Hour,Offense Type,Beat,Premise,Block Range,Street Name,Type,Suffix,Offenses,BlockRange,StreetName,# offenses,# Of Offenses,# Offenses,# Of
0,01/16/2017,5,Theft,10H10,Commercial Parking Lot or Garage,6000-6099,CANAL,ST,-,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,03/31/2017,18,Rape,10H10,Residence or House,200-299,N LENOX ST,ST,-,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,09/03/2017,19,Burglary,10H10,Apartment,1900-1999,RUNNELS,-,-,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df2.head(3)

,Occurrence Date,Occurrence Hour,NIBRS Description,Offense Count,Beat,Premise,Block Range,Street Name,Street Type,Suffix
NaN,2018-09-01,0,"Credit card, ATM fraud",1,5F30,Auto Dealership New/Used,12800-12899,HEMPSTEAD,RD,NaN
NaN,2018-09-01,0,Intimidation,1,8C30,"Residence, Home (Includes Apartment)",8100-8199,SNOWDEN,NaN,NaN
NaN,2018-09-01,0,Weapon law violations,1,14D10,"Parking Lot, Garage",6600-6699,MADRID,ST,NaN


### **6. Save Dataframes to CSV Files**

In [ ]:
# Save df, df2, df3 as jan2010_may2018.csv, jun2018_dec2018.csv and jan2019_jul2023.csv respectively
df.to_csv('data/jan2010_may2018.csv', index=False)
df2.to_csv('data/jun2018_dec2018.csv', index=False)
df3.to_csv('data/jan2019_jul2023.csv', index=False)

### **7. Load CSV Files**

In [ ]:
# Check jan2010_may2018.csv, jun2018_dec2018.csv and jan2019_jul2023.csv
df_jan2010_may2018 = pd.read_csv('data/jan2010_may2018.csv', dtype={'Beat': str, 'Block Range':str})
df_jun2018_dec2018 = pd.read_csv('data/jun2018_dec2018.csv', dtype={'Beat': str, 'Block Range':str})
df_jan2019_jul2023 = pd.read_csv('data/jan2019_jul2023.csv', dtype={'Beat': str})

In [ ]:
print("df_jan2010_may2018", df_jan2010_may2018.shape)
print("df_jun2018_dec2018", df_jun2018_dec2018.shape)
print("df_jan2019_jul2023", df_jan2019_jul2023.shape)

In [ ]:
# Check duplicates
print("df_jan2010_may2018", df_jan2010_may2018.duplicated().sum())
print("df_jun2018_dec2018", df_jun2018_dec2018.duplicated().sum())
print("df_jan2019_jul2023", df_jan2019_jul2023.duplicated().sum())

In [ ]:
# Check jan2010_may2018.csv
df_jan2010_may2018.head()

In [ ]:
# Check df_jun2018_dec2018
df_jun2018_dec2018.head()

In [ ]:
# Check df_jun2018_dec2018
df_jan2019_jul2023.head()